# Simple LangGraph Agent: From LLMs to Agents

## 1. LLMs vs Agents: Key Differences 🤔

### Traditional LLMs
- **Input → Output**: Simple text-in, text-out pattern
- **Static**: No ability to interact with external systems
- **Single Turn**: One response per query
- **Limited Context**: Only what's in the prompt
- **No Tools**: Cannot perform actual calculations or actions

### AI Agents
- **Interactive**: Can use tools and external APIs
- **Multi-step**: Break down complex tasks into steps
- **Memory**: Maintain state across interactions
- **Planning**: Can reason about next actions
- **Error Handling**: Can retry and recover from failures
- **Tool Usage**: Can perform real actions (calculations, API calls, etc.)

### Visual Comparison

```
LLM Pattern:
User: "What is 15 × 23?"
LLM: "15 × 23 equals 345" (might be wrong!)

Agent Pattern:
User: "What is 15 × 23?"
Agent: 1. Detects math needed
        2. Uses calculator tool  
        3. Returns: "The answer is: 345" (always correct!)
```

## This Demo Shows:
- **Chat Mode**: "How are you?" → Normal conversation
- **Tool Mode**: "What is 5 + 3?" → Uses calculator tool
- **Smart Routing**: Automatically detects what type of response needed

In [ ]:
# Install required packages
!pip install langgraph langchain requests

In [ ]:
import re
import requests
from typing import Dict, Any
from langgraph.graph import StateGraph, END
from langchain.tools import tool

# 2. LangGraph Core Components 📊

print("🔍 LangGraph is built around several key concepts:")
print()
print("🔄 STATE")
print("   • Purpose: Maintains information throughout agent execution")
print("   • Type: Dictionary that defines the data structure") 
print("   • Persistence: Carries context between different steps")
print()
print("📊 NODES") 
print("   • Purpose: Individual functions that perform specific tasks")
print("   • Input: Receive the current state")
print("   • Output: Return updates to the state")
print("   • Examples: LLM calls, tool usage, data processing")
print()
print("➡️ EDGES")
print("   • Purpose: Define the flow between nodes")
print("   • Types: Normal edges (direct) & Conditional edges (dynamic routing)")
print()
print("🔧 TOOLS")
print("   • Purpose: External functions the agent can call")
print("   • Examples: Web search, calculators, APIs, databases")
print("   • Integration: Seamlessly integrated into agent workflow")

In [ ]:
# E2E Networks API Configuration
E2E_API_URL = "https://api.e2enetworks.com/v1/chat/completions"  # Update with actual endpoint
E2E_API_KEY = "your-e2e-api-key"  # Replace with your actual API key

def call_e2e_llm(message: str) -> str:
    """Call E2E Networks LLM API"""
    headers = {
        "Authorization": f"Bearer {E2E_API_KEY}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "model": "gpt-3.5-turbo",  # Update with E2E model name
        "messages": [{"role": "user", "content": message}],
        "max_tokens": 150,
        "temperature": 0.7
    }
    
    try:
        response = requests.post(E2E_API_URL, headers=headers, json=payload)
        if response.status_code == 200:
            return response.json()["choices"][0]["message"]["content"]
        else:
            return f"Error: {response.status_code}"
    except Exception as e:
        return f"Error calling E2E API: {str(e)}"

print("🔧 E2E Networks API configured")

In [ ]:
# Simple calculator tool
@tool
def calculator(expression: str) -> str:
    """Calculate mathematical expressions like 5+3, 10*2, etc."""
    try:
        # Clean the expression
        expression = expression.replace(" ", "")
        
        # Only allow safe characters
        allowed = set('0123456789+-*/.() ')
        if not all(c in allowed for c in expression):
            return "Invalid expression"
        
        result = eval(expression)
        return str(result)
    except:
        return "Error in calculation"

print("🧮 Calculator tool ready")
print("🎯 This is what makes it an AGENT - it can use TOOLS!")
print("📝 LLM alone: Can only generate text about math")
print("🤖 Agent: Can actually CALCULATE math using tools")

In [ ]:
# Simple state (just a dictionary)
def create_state(user_input: str) -> Dict[str, Any]:
    return {
        "user_input": user_input,
        "needs_math": False,
        "result": ""
    }

# Check if input needs math calculation
def check_math_need(state: Dict) -> Dict:
    user_input = state["user_input"].lower()
    
    # Look for math patterns
    math_keywords = ["calculate", "what is", "+", "-", "*", "/", "plus", "minus", "times"]
    has_numbers = bool(re.search(r'\d', user_input))
    has_math_words = any(word in user_input for word in math_keywords)
    
    state["needs_math"] = has_numbers and has_math_words
    return state

# Do math calculation
def do_math(state: Dict) -> Dict:
    user_input = state["user_input"]
    
    # Extract numbers and operators
    math_pattern = r'[0-9+\-*/().\s]+'
    matches = re.findall(math_pattern, user_input)
    
    if matches:
        expression = matches[0].strip()
        result = calculator.invoke({"expression": expression})
        state["result"] = f"The answer is: {result}"
    else:
        state["result"] = "I couldn't find a math expression to calculate."
    
    return state

# Normal chat response - CLEANED UP VERSION
def chat_response(state: Dict) -> Dict:
    user_input = state["user_input"].lower()
    
    # Define response patterns
    responses = {
        "how are you": "I'm doing great! I'm a helpful AI agent that can chat and do math calculations.",
        "who are you": "I'm a LangGraph agent powered by E2E Networks. I can help with conversations and mathematical calculations!",
        "hello": "Hello! How can I help you today? I can chat or help with math calculations.",
        "hi": "Hi there! How can I help you today? I can chat or help with math calculations."
    }
    
    # Check for predefined responses
    for pattern, response in responses.items():
        if pattern in user_input:
            state["result"] = response
            return state
    
    # For other questions, use E2E LLM
    state["result"] = call_e2e_llm(state["user_input"])
    return state

# Route to math or chat
def route_request(state: Dict) -> str:
    return "do_math" if state["needs_math"] else "chat_response"

print("🤖 Agent functions ready")
print("✅ Chat response function cleaned up - no more multiple if-else!")

In [ ]:
# Create the simple agent
workflow = StateGraph(dict)

# Add nodes
workflow.add_node("check_math", check_math_need)
workflow.add_node("do_math", do_math)
workflow.add_node("chat_response", chat_response)

# Set entry point
workflow.set_entry_point("check_math")

# Add conditional routing
workflow.add_conditional_edges(
    "check_math",
    route_request,
    {
        "do_math": "do_math",
        "chat_response": "chat_response"
    }
)

# Both end at END
workflow.add_edge("do_math", END)
workflow.add_edge("chat_response", END)

# Compile the agent
agent = workflow.compile()
print("✅ Simple agent created!")
print()
print("🔄 Agent Workflow:")
print("1. check_math → Analyzes if math is needed")
print("2a. do_math → Uses calculator tool (if math)")  
print("2b. chat_response → Normal conversation (if not math)")
print("3. END → Returns result")
print()
print("🧠 This is REASONING - the agent decides which path to take!")

In [ ]:
# Test the agent
def ask_agent(question: str):
    print(f"\n👤 You: {question}")
    
    state = create_state(question)
    result = agent.invoke(state)
    
    print(f"🤖 Agent: {result['result']}")
    return result

# Test cases
test_questions = [
    "How are you?",
    "Who are you?", 
    "What is 5 + 3?",
    "Calculate 10 * 2",
    "Hello there!",
    "What is 15 - 7?"
]

print("🧪 Testing the agent...")
for question in test_questions:
    ask_agent(question)

In [ ]:
# Interactive mode
print("\n💬 Interactive Mode - Type 'quit' to exit")
print("Try asking: 'How are you?', 'What is 20 + 15?', 'Calculate 100 / 5'")

while True:
    user_question = input("\n👤 You: ")
    
    if user_question.lower() in ['quit', 'exit', 'bye']:
        print("🤖 Agent: Goodbye!")
        break
    
    if user_question.strip():
        ask_agent(user_question)

## How It Works - The Agent Flow

### 🔍 Step-by-Step Execution:

1. **Input Analysis**: Checks if the question needs math calculation
2. **Smart Routing**: Routes to either math calculator or chat response  
3. **Processing**: 
   - **Math Path**: Uses calculator tool
   - **Chat Path**: Uses simple responses or E2E Networks API
4. **Response**: Returns the result

### 🎯 Key Agent Capabilities Demonstrated:

✅ **Tool Usage** - Can perform actual calculations  
✅ **Decision Making** - Routes based on input type  
✅ **State Management** - Tracks conversation context  
✅ **Multi-modal Response** - Handles both chat and computation  

### 📈 Why This Matters:

- **LLM**: "What's 15 * 23?" → "15 * 23 = 345" (might be wrong!)
- **Agent**: "What's 15 * 23?" → Uses calculator → "The answer is: 345" (always correct!)

**Simple and Clean!** 🎯